In [ ]:
from pathlib import Path
from typing import List, Tuple, Optional, Dict
import logging, json, re, random

import pandas as pd
import numpy as np
import pydicom                       # low-level header access (optional)
import nrrd                          # pynrrd
import SimpleITK as sitk             # high-level I/O
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

logging.basicConfig(level=logging.INFO,
                    format="%(levelname)s | %(message)s")

DATA_ROOT = Path("/home/mariopasc/Python/Datasets/Meningiomas/men")
FLAIR_DIRNAME = "flair"              # change here if convention differs
TARGET_EXT = ".nrrd"

from pathlib import Path
import flair2nrrd

root   = Path("/home/mariopasc/Python/Datasets/Meningiomas/men")
outdir = root.parent / "Derived"          # wherever you like

df = flair2nrrd.batch_convert_flair(root, outdir)
df.head()


/home/mariopasc/Python/misc/miniconda3/envs/meningiomas/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
